# Word chains

"Word chain" puzzles are where you transform one word into another, by changing one letter at a time, with all the intermediate steps being valid words. 

For instance, you can transform 'rash' to 'jags' like this:

```
rash
Bash
basS
baGs
Jags
```

(the capital letter is the one changed in each step).

## Part 1

Given this [list of words](words4.txt), what is the minimum number of steps to go from `vice` to `wars`?

In [1]:
import string
import heapq

In [2]:
words = [w.strip() for w in open('words4.txt').readlines()]
len(words)

2336

In [3]:
def adjacents(word):
    return [word[0:i] + l + word[i+1:]
           for i in range(len(word))
           for l in string.ascii_lowercase
           if l != word[i]]

In [4]:
neighbours = {w: [n for n in adjacents(w) if n in words]
             for w in words}

In [5]:
def distance(w1, w2):
    return sum(1 for i in range(len(w1))
               if w1[i] != w2[i])

In [6]:
# def extend(chain):
#     return [chain + [s] for s in neighbours[chain[-1]]
#            if s not in chain]

In [7]:
def extend(chain, closed=None):
    if closed:
        nbrs = set(neighbours[chain[-1]]) - closed
    else:
        nbrs = neighbours[chain[-1]]
    return [chain + [s] for s in nbrs
           if s not in chain]

In [8]:
def extend_raw(chain):
    nbrs = [w for w in adjacents(chain[-1]) if w in words]
    return [chain + [s] for s in nbrs
           if s not in chain]

In [9]:
def bfs_search(start, goal, debug=False):
    agenda = [[start]]
    finished = False
    while not finished and agenda:
        current = agenda[0]
        if debug:
            print(current)
        if current[-1] == goal:
            finished = True
        else:
            successors = extend(current)
            agenda = agenda[1:] + successors
    if agenda:
        return current
    else:
        return None        

In [19]:
def bfs_search_closed(start, goal, debug=False):
    agenda = [[start]]
    closed = set()
    finished = False
    while not finished and agenda:
        current = agenda[0]
        if debug:
            print(current)
        if current[-1] == goal:
            finished = True
        else:
            closed.add(current[-1])
            successors = extend(current, closed)
            agenda = agenda[1:] + successors
    if agenda:
        return current
    else:
        return None   

In [10]:
def dfs_search(start, goal, debug=False):
    agenda = [[start]]
    finished = False
    while not finished and agenda:
        current = agenda[0]
        if debug:
            print(current)
        if current[-1] == goal:
            finished = True
        else:
            successors = extend(current)
            agenda = successors + agenda[1:]
    if agenda:
        return current
    else:
        return None        

In [11]:
def astar_search(start, goal, debug=False):
    agenda = [(distance(start, goal), [start])]
    heapq.heapify(agenda)
    finished = False
    while not finished and agenda:
        _, current = heapq.heappop(agenda)
        if debug:
            print(current)
        if current[-1] == goal:
            finished = True
        else:
            successors = extend(current)
            for s in successors:
                heapq.heappush(agenda, (len(current) + distance(s[-1], goal) - 1, s))
    if agenda:
        return current
    else:
        return None        

In [12]:
# Uses direct lookup of successors, rather than using cached neighbours in the dict
def astar_search_raw(start, goal, debug=False):
    agenda = [(distance(start, goal), [start])]
    heapq.heapify(agenda)
    finished = False
    while not finished and agenda:
        _, current = heapq.heappop(agenda)
        if debug:
            print(current)
        if current[-1] == goal:
            finished = True
        else:
            successors = extend_raw(current) # Difference here
            for s in successors:
                heapq.heappush(agenda, (len(current) + distance(s[-1], goal) - 1, s))
    if agenda:
        return current
    else:
        return None        

In [13]:
def astar_search_closed(start, goal, debug=False):
    agenda = [(distance(start, goal), [start])]
    heapq.heapify(agenda)
    closed = set()
    finished = False
    while not finished and agenda:
        _, current = heapq.heappop(agenda)
        if debug:
            print(current)
        if current[-1] == goal:
            finished = True
        else:
            closed.add(current[-1])
            successors = extend(current, closed)
            for s in successors:
                heapq.heappush(agenda, (len(current) + distance(s[-1], goal) - 1, s))
    if agenda:
        return current
    else:
        return None   

In [14]:
astar_search('vice', 'wars')

['vice', 'dice', 'dire', 'dare', 'ware', 'wars']

In [15]:
astar_search_raw('vice', 'wars')

['vice', 'dice', 'dire', 'dare', 'ware', 'wars']

In [16]:
len(astar_search('vice', 'wars'))

6

In [17]:
len(bfs_search('vice', 'wars'))

6

In [20]:
len(bfs_search_closed('vice', 'wars'))

6

In [21]:
len(dfs_search('vice', 'wars'))

793

In [22]:
%%timeit
astar_search('vice', 'wars')

10000 loops, best of 3: 158 µs per loop


In [23]:
%%timeit
astar_search_raw('vice', 'wars')

100 loops, best of 3: 15.6 ms per loop


In [24]:
%%timeit
astar_search_closed('vice', 'wars')

10000 loops, best of 3: 168 µs per loop


In [25]:
%%timeit
bfs_search('vice', 'wars')

1 loop, best of 3: 1min 40s per loop


In [26]:
%%timeit
bfs_search_closed('vice', 'wars')

1 loop, best of 3: 597 ms per loop


In [27]:
%%timeit
dfs_search('vice', 'wars')

10 loops, best of 3: 85.5 ms per loop


## Part 2

The example shows that `jags` is reachable in four steps from `rash`. There are 11 words one step away from `rash`: 
`bash`, `cash`, `dash`, `gash`, `hash`, `lash`, `mash`, `rasp`, `rush`, `sash`, and `wash`. 

There are 47 words reachable in one or two steps from `rash`. They are `base`, `bash`, `bask`, `bass`, `bast`, `bath`, `bosh`, `bush`, `case`, `cash`, `cask`, `cast`, `dash`, `dish`, `gash`, `gasp`, `gosh`, `gush`, `hash`, `hasp`, `hath`, `hush`, `lash`, `lass`, `last`, `lath`, `lush`, `mash`, `mask`, `mass`, `mast`, `math`, `mesh`, `mush`, `push`, `ramp`, `rasp`, `ruse`, `rush`, `rusk`, `rust`, `sash`, `sass`, `tush`, `wash`, `wasp`, and `wish`.

There are 180 words reachable in up to three steps from `rash`.

How many words are reachable in up to ten steps from `vice`?

In [28]:
def reachable_in(word, n, trim_extras=False):
    reachable = set()
    boundary = set([word])
    for i in range(n):
        extras = set()
        for w in boundary:
            extras.update(neighbours[w])
        if trim_extras:
            extras.difference_update(reachable)
        reachable.update(boundary)
        boundary = extras.copy()
    return reachable.union(extras).difference(set([word]))

In [29]:
len(reachable_in('rash', 1)), ', '.join(sorted('`{}`'.format(r) for r in reachable_in('rash', 1)))

(11,
 '`bash`, `cash`, `dash`, `gash`, `hash`, `lash`, `mash`, `rasp`, `rush`, `sash`, `wash`')

In [30]:
len(reachable_in('rash', 2)), ', '.join(sorted('`{}`'.format(r) for r in reachable_in('rash', 2)))

(47,
 '`base`, `bash`, `bask`, `bass`, `bast`, `bath`, `bosh`, `bush`, `case`, `cash`, `cask`, `cast`, `dash`, `dish`, `gash`, `gasp`, `gosh`, `gush`, `hash`, `hasp`, `hath`, `hush`, `lash`, `lass`, `last`, `lath`, `lush`, `mash`, `mask`, `mass`, `mast`, `math`, `mesh`, `mush`, `push`, `ramp`, `rasp`, `ruse`, `rush`, `rusk`, `rust`, `sash`, `sass`, `tush`, `wash`, `wasp`, `wish`')

In [31]:
len(reachable_in('rash', 3))

180

In [32]:
len(reachable_in('rash', 10))

2195

In [33]:
len(reachable_in('vice', 10))

2192

In [34]:
%%timeit
len(reachable_in('rash', 10))

100 loops, best of 3: 5.82 ms per loop


In [35]:
%%timeit
len(reachable_in('rash', 10, trim_extras=True))

100 loops, best of 3: 2.75 ms per loop
